# 01 路径参数

In [ ]:
# 小demo
import uvicorn
from fastapi import FastAPI

app = FastAPI()
@app.get("/")
async def read_root():
    return {"message": "Hello, World!"}

if __name__ == "__main__":
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()


In [ ]:
# 路径参数
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/items/{item_id}")
async def read_item(item_id:int):
    return{"item_id": item_id}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()
"""
访问 http://127.0.0.1:8000/items/3
会看到{"item_id":3}

这将为你的函数提供编辑器支持，包括错误检查、代码补全等等。

但如果你通过浏览器访问 http://127.0.0.1:8000/items/foo，你会看到一个清晰可读的 HTTP 错误，因为foo是string，而不是int

如果你提供的是 float 而非整数也会出现同样的错误，比如： http://127.0.0.1:8000/items/4.2
"""

#### 预设值
如果你有一个**接收路径参数**的路径操作，但你希望预先设定可能的有效参数值，则可以使用标准的 Python Enum 类型。

创建一个预定义的类 继承自 str 和 Enum 。

通过从 str 继承，API 文档将能够知道这些值必须为 string 类型并且能够正确地展示出来。

In [ ]:
from fastapi import FastAPI
inport uvicorn 
from enum import Enum

app = FastAPI()

class ModelName(str,Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet" 

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name == ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}
    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}
    return {"model_name": model_name, "message": "Have some residuals"}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

#### 包含路径的路径参数
主要是因为路径里面的反斜杠 ‘/’ 跟http里的一样，可能会被解读成http的反斜杠

使用直接来自 Starlette 的选项来声明一个包含路径的路径参数：

/files/{file_path:path}

在这种情况下，参数的名称为 file_path，结尾部分的 :path 说明该参数应匹配任意的路径。

In [ ]:
from fastapi import FastAPI
import uvicorn 

app = FastAPI()

@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

"""
打开浏览器，输入 http://127.0.0.1:8000/files/home/johndoe/myfile.txt

会看到： {"file_path":"home/johndoe/myfile.txt"}
"""

# 查询参数
声明不属于路径参数的其他函数参数时，它们将被自动解释为"查询字符串"参数

查询字符串是键值对的集合，这些键值对位于 URL 的 ？ 之后，并以 & 符号分隔。

例如：

http://127.0.0.1:8000/items/?skip=0&limit=10  问号 ？ 后的就是查询参数 skip 和 limit，用 & 符号分割

skip：对应的值为 0 limit：对应的值为 10

In [ ]:
from fastapi import FastAPI
import uvicorn

db = [
    {"item_name": "Foo"},
    {"item_name": "GPU"},
    {"item_name": "ROG"},]

app = FastAPI()

@app.get("/items/")
async def read_items(skip: int = 0, limit: int = 10):  # 这里skip默认值为0，limit默认值为10
    return db[skip : skip + limit]  # 切片操作 [0:10]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [26876]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59617 - "GET /items/3 HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59617 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59617 - "GET /items HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:59617 - "GET /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [26876]


#### 必须 查询参数
async def read_items(skip: int = 0, limit: int = 10)，为查询参数设置默认值意味着它不是必须的

如果要让用户必须输入查询参数，则只要声明但是不设置默认值

In [ ]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_user_item(item_id: str, needy: str):
    item = {"item_id": item_id, "needy": needy}
    return item

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

"""
这里的查询参数 needy 是类型为 str 的必需查询参数。

浏览器输入 http://127.0.0.1:8000/items/4?needy=yes

会看{"item_id":"4","needy":"yes"}

如果浏览器输入 http://127.0.0.1:8000/items/4

会报错 {"detail":[{"type":"missing","loc":["query","needy"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.5/v/missing"}]}"}
"""

#### 可选的查询参数
通过设置成 var:Union[None|str] = None 就可以变成可选的，意思是 None或者str

In [ ]:
import uvicorn
from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
async def read_item(item_id: str, q: Union[str, None] = None):
    if q:
        return {"item_id": item_id, "q": q}
    return {"item_id": item_id}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

"""
浏览器输入 http://127.0.0.1:8000/items/4

会看到 {"item_id":"4"}

输入 http://127.0.0.1:8000/items/4?q=hello

会看到 {"item_id":"4","q":"hello"}
"""

# 路径参数校验
可以使用 Path 为路径参数声明类型的校验和元数据。

导入 Path

首先，从 fastapi 导入 Path：

声明元数据，尽可能选择使用 Annotated 语法。

In [ ]:
import uvicorn
from fastapi import FastAPI, Path
from typing import Annotated

app = FastAPI()

@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[int, Path(title="The ID of the item to get")],
):
    return {"item_id": item_id}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()
"""
浏览器中输入 http://127.0.0.1:8000/items/5

会看到 {"item_id":5}
"""

#### 数值校验
添加 ge=5.5 后，item_id 将必须是一个大于（greater than）或等于（equal）5.5 的数

gt：大于（greater tha n）

ge：大于等于（greater than or equ al）

lt：小于（less t han）

le：小于等于（less than or equal）

In [2]:
import uvicorn
from typing import Annotated
from fastapi import FastAPI, Path
app = FastAPI()
@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[float, Path(title="The ID of the item to get", ge=5.5)]
):
    results = {"item_id": item_id}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [26876]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [26876]


# 查询参数校验 Query
除了可选与必须的约束外

我们打算添加约束条件：即使查询参数 q 是可选的，但只要提供了该参数，则该参数值不能超过5个字符的长度。

为此，首先从 fastapi 导入 Query：

现在，将 Query 用作查询参数的默认值，并将它的 max_length 参数设置为 5：

In [ ]:
from fastapi import FastAPI, Query
from typing import Union, Annotated
import uvicorn

app = FastAPI()

@app.get("/items/")
async def read_items(q: Annotated[Union[str, None], Query(default=None, max_length=5)]e):
# 也可以不用 Annotated
#async def read_items(q: Union[str, None] = Query(default=None, max_length  =5)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

你可以添加更多有关该参数的信息。

可以添加正则表达式

你可以添加 title：

Query(default=None, title="Query string", min_length=3)

以及 description：

Query(default=None, title="Query string", description="Query string for the items to search in the database", min_length=3)

# 请求体参数
参数有三种类型：路径参数、查询参数和请求体参数。

如果一个参数是url路径中的参数，那么这个参数就会被默认是路径参数。

如果一个单个参数不是url路径中的参数，那么这个参数就会被默认是查询参数。即便是用post方法，参数也会被默认是查询参数。

如果一个参数的类型是一个**pydantic模型**，那么这个参数就会被默认是请求体参数。

In [ ]:
# 请求体参数必须用 Body 声明，否则会被当作查询参数处理
from fastapi import FastAPI, Body
from typing import Annotated
import uvicorn
app = FastAPI()
@app.post("/items/")
async def create_item(item: Annotated[int,Body()]):
    return {"item": item}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
请求体意味着用json格式发送数据，而不是直接在url里拼接
import requests

url = "http://127.0.0.1:8009/items/"
response = requests.post(url,json=5)
print(response.text)

输出 {"item":5}
"""

INFO:     Started server process [25208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:53628 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53629 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53647 - "POST /items/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53749 - "GET /items/5 HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53749 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53750 - "GET /items?5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:53750 - "GET /items/?5 HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:58792 - "GET /items/?item=5 HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25208]


#### 多个请求体 Body()
差别在传入时，可以用字典定义多个要传入的请求体

In [ ]:
from fastapi import FastAPI, Body
from typing import Annotated
import uvicorn
app = FastAPI()
@app.post("/items/")
async def get_data(
    item: Annotated[int, Body()],
    name: Annotated[str, Body()],
):
    return {"item": item, "name": name}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
import requests
url = 
data = {"item": 5,
        "name": "foo"}           用字典定义多个请求信息
response = requests.post(url,json=data)
"""

#### Pydantic模型参数
用于校验对应类型的参数是否符合自定义的要求

同样的，也需要在requests里传入字典

In [ ]:
from fastapi import FastAPI
import uvicorn
from pydantic import BaseModel
app = FastAPI()

class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

@app.post("/items/")
async def create_item(item: Item):
    return item

""""
url = 
data = {
    "name": "Foo",
    "description": "A very nice Item",
    "price": 35.4,
    "tax": 3.2,
}
response = requests.post(url,json=data)
print(response.text)
"""

#### 多个Pydantic 
fastapi 会自动用记住对应参数的名字

传入的时候构建字典，用对应参数名字作为键

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()
class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None

class User(BaseModel):
    username: str
    full_name: Union[str, None] = None

@app.post("/items/")
async def create_item(item: Item, user: User):
    return {"item": item, "user": user}
if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8000)
    server = uvicorn.Server(config)
    await server.run()

"""
url = 
data={
    "item": {
        "name": "Foo",
        "description": "A very nice Item",
        "price": 35.4,
        "tax": 3.2,
    },
    "user": {
        "username": "johndoe",
        "full_name": "John Doe",
    },
}
"""

#### 嵌入单个请求体参数 Body()+pydantic
对于多个pydantic，fastapi会记住参数名字，用户可以将参数名字作为键来构建data字典

但是对于单个pydantic，可以使用 Body(embed=True), 使得单个pydantic的情况下，用户也可以使用参数名字作为键来构建data字典。

In [ ]:
from fastapi import FastAPI, Body
from pydantic import BaseModel
import uvicorn
app = FastAPI()
class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None
@app.post("/items/")
async def create_item(item: Item = Body(embed=True)): # 使用embed=True 
    return item

"""
url =
data = {
    "item": {               使用embed=True , 即使是单个pydantic模型也可以使用参数名来作为键
        "name": "Foo",
        "description": "A very nice Item",
        "price": 35.4,
        "tax": 3.2,
    },
}
"""